In [94]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [132]:
### variables for dota2 version 1.0
# game = 'dota2'
# redone_filter = ['Push', 'Split Push', 'Roshan']
# titles = ['Push', 'Split Push', 'Roshan', 'No']
# heading = ['Push', 'Split Push', 'Roshan', 'No Label', 'TimeStamp', 'Push', 'Split Push', 'Roshan', 'No Label', 'TimeStamp']
# rename_title = {}
# t_hold_same = 0
# t_hold_diff = 0

# ### variables for dota2 version 2.0
# game = 'dota2'
# redone_filter = ['Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push']
# titles = ['Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push', 'No']
# heading = ['Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push', 'No Label', 'TimeStamp', 'Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push', 'No Label', 'TimeStamp']
# rename_title = {}
# t_hold_same = 0
# t_hold_diff = 0


game = 'boomtown'
redone_filter = ['Explosive Inner', 'Explosive Quick Mine', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss']
titles = ['Explosive Quick Mine', 'Explosive Inner', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No']
# heading = ['timepoint', 'Explosive Inner', 'Explosive Quick Mine', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No/Other Label', 'Explosive Inner', 'Explosive Quick Mine', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No/Other Label']
# heading = ['Jimmy Explosive Quick Mine', 'Jimmy Explosive Inner', 'Jimmy Explosive Tunneling', 'Jimmy Exploring', 'Jimmy Tunneling Pickaxe', 'Jimmy Pickaxing for Gold', 'Jimmy Explosive Miss', 'Jimmy No label', 'Jimmy TimeStamp', 'Erica Explosive Quick Mine', 'Erica Explosive Inner', 'Erica Explosive Tunneling', 'Erica Exploring', 'Erica Tunneling Pickaxe', 'Erica Pickaxing for Gold', 'Erica Explosive Miss', 'Erica No label', 'Erica TimeStamp']
heading = ['Explosive Quick Mine', 'Explosive Inner', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No Label', 'TimeStamp', 'Explosive Quick Mine', 'Explosive Inner', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No Label', 'TimeStamp']
rename_title = {'Explosive - Inner': 'Explosive Inner', 'Tunneling - Explosive' :'Explosive Tunneling'}
t_hold_same = 2
t_hold_diff = 2

original_label_dir = './data/'+game+'/original/'
redone_label_dir = './data/'+game+'/redone/'

orginal_label_initial = 'Erica'  # Original
redone_label_initial = 'Jimmy'  # Redone

irr_table_output_file = './data/'+game+'/'+game+'_irr_table.csv'
irr_matrix_output_file = './data/'+game+'/'+game+'_irr_matrix.csv'

In [133]:
def get_name_and_extension(file_name):
    """
    get name and extension of a file_name
    :param file_name:
    :return:
    """
    tokens = file_name.split('.')
    return {'name': '.'.join(tokens[0:-1]), 'ext': tokens[-1]}

In [134]:
def check_overlap(t1, t2):
    s1 = t1[0]
    f1 = t1[1]
    s2 = t2[0]
    f2 = t2[1]
#     print(s1, f1, s2, f2)
    
    if s2<=s1 and f1<=f2:
#         print('c1')
        return f1-s1+1
    elif s1<=s2 and f2<=f1:
#         print('c2')
        return f2-s2+1
    elif s2<=s1 and f2<=f1 and s1<=f2:
#         print('c3')
        return f2-s1+1
    elif s1<=s2 and f1<=f2 and s2<=f1:
#         print('c4')
        return f1-s2+1
    
    return -1

In [135]:
# ## previous version
# def analyze_single_player(original_labels, redone_labels, table_info):
    
#     # only looking at same titles
#     for label in redone_labels:
# #         print(label['title'], label['timestamp_range'], label['author'], label['units'])
#         label['longest'] = -1
#         for ori_label in original_labels:
#             overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
#             if  overlap > -1:
# #                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
#                 temp_title = ori_label['title']
#                 if ori_label['title'] in rename_title:    
#                     temp_title = rename_title[ori_label['title']]
                
#                 if temp_title == label['title'] and overlap>t_hold_same:
# #                     print('----FOUND----')
#                     ori_label['checked'] = True
#                     label['checked'] = True
#                     table_info.append((label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range']))
#                     print(0, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
#                     if label['longest'] < overlap:
#                         label['longest'] = overlap
    
#     # looking at different titles
#     for label in redone_labels:
# #         print(label['title'], label['timestamp_range'], label['author'], label['units'])
#         if 'checked' in label:
#             for ori_label in original_labels:
#                 if label['title'] != ori_label['title']:
#                     overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
#                     if overlap > label['longest'] and overlap > t_hold_diff:
#                         temp_title = ori_label['title']
#                         if ori_label['title'] in rename_title:    
#                             temp_title = rename_title[ori_label['title']]
#                         ori_label['checked'] = True
#                         table_info.append((label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range']))
#                         print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
#                         print('S/D', overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
#                         print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
#             continue
        
#         for ori_label in original_labels:
#             overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
#             if  overlap > -1:
# #                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
#                 temp_title = ori_label['title']
#                 if ori_label['title'] in rename_title:    
#                     temp_title = rename_title[ori_label['title']]

#                 if 'checked' not in ori_label and temp_title in redone_filter and overlap>t_hold_diff:
#                     ori_label['checked'] = True
#                     label['checked'] = True
#                     table_info.append((label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range']))
#                     print(1, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
#                     break

        
#     # generate No labels for unchecked redone_labels
#     for label in redone_labels:
#         if 'checked' not in label:
#             table_info.append((label['title'], 'No', label['timestamp_range'], -1))
#             print(2, '[', label['title'], 'No', label['timestamp_range'], -1, ']')

#     # generate No labels for unchecked original_labels
#     for ori_label in original_labels:
#         temp_title = ori_label['title']
#         if ori_label['title'] in rename_title:    
#             temp_title = rename_title[ori_label['title']]
            
#         if temp_title in redone_filter:
# #             if 'checked' in ori_label:
# #                 print(ori_label)
#             if 'checked' not in ori_label:
#                 table_info.append(('No', temp_title, -1, ori_label['timestamp_range']))
#                 print(3, '[', 'No', temp_title, -1, ori_label['timestamp_range'], ']')
                

In [136]:
def analyze_single_player(original_labels, redone_labels, table_info):
    
    # only looking at same titles
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        label['longest'] = -1
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]
                
                if temp_title == label['title'] and overlap>t_hold_same:
#                     print('----FOUND----')
                    ori_label['checked'] = True
                    label['checked'] = True
                    table_info.append((label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range']))
                    print(0, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
                    if label['longest'] < overlap:
                        label['longest'] = overlap
    
    # looking at different titles
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        if 'checked' in label:
            i = 0
            longest = -1
            longest_index = -1
            for ori_label in original_labels:
                if label['title'] != ori_label['title']:
                    overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
                    if overlap > label['longest'] and overlap > t_hold_diff:
                        if overlap > longest:
                            longest = overlap
                            longest_index = i
                        
                i += 1
            
            if longest_index > -1:
                temp_title = original_labels[longest_index]['title']
                if original_labels[longest_index]['title'] in rename_title:    
                    temp_title = rename_title[original_labels[longest_index]['title']]
                    
                original_labels[longest_index]['checked'] = True
                table_info.append((label['title'], temp_title, label['timestamp_range'], original_labels[longest_index]['timestamp_range']))
                print('S/D', '[', label['title'], temp_title, label['timestamp_range'], original_labels[longest_index]['timestamp_range'], ']')

            continue
        
        i = 0
        longest = -1
        longest_index = -1
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]

                if 'checked' not in ori_label and temp_title in redone_filter and overlap>t_hold_diff:
                    if overlap > longest:
                        longest = overlap
                        longest_index = i

            i += 1
        
        if longest_index > -1:
            temp_title = original_labels[longest_index]['title']
            if original_labels[longest_index]['title'] in rename_title:    
                temp_title = rename_title[original_labels[longest_index]['title']]
                    
            original_labels[longest_index]['checked'] = True
            label['checked'] = True
            table_info.append((label['title'], temp_title, label['timestamp_range'], original_labels[longest_index]['timestamp_range']))
            print(1, '[', label['title'], temp_title, label['timestamp_range'], original_labels[longest_index]['timestamp_range'], ']')

        
    # generate No labels for unchecked redone_labels
    for label in redone_labels:
        if 'checked' not in label:
            table_info.append((label['title'], 'No', label['timestamp_range'], -1))
            print(2, '[', label['title'], 'No', label['timestamp_range'], -1, ']')

    # generate No labels for unchecked original_labels
    for ori_label in original_labels:
        temp_title = ori_label['title']
        if ori_label['title'] in rename_title:    
            temp_title = rename_title[ori_label['title']]
            
        if temp_title in redone_filter:
#             if 'checked' in ori_label:
#                 print(ori_label)
            if 'checked' not in ori_label:
                table_info.append(('No', temp_title, -1, ori_label['timestamp_range']))
                print(3, '[', 'No', temp_title, -1, ori_label['timestamp_range'], ']')
                

In [137]:
def separate_players(labels):
    player_labels = {}
    for label in labels:
        for unit in label['units']:
            if unit not in player_labels:
                player_labels[unit] = [label]
            else:
                player_labels[unit].append(label)
    return player_labels

In [138]:
def analyze_labels_by_players(original_labels, redone_labels, table_info):
    # filtering
    redone_labels_filtered = []
    for label in redone_labels:
        temp_title = label['title']
        if label['title'] in rename_title:    
            temp_title = rename_title[label['title']]
            
        if temp_title in redone_filter:
            redone_labels_filtered.append(label)
            
    original_player_labels = separate_players(original_labels)
    redone_player_labels = separate_players(redone_labels_filtered)
    
    for pl in redone_player_labels:
        if pl in original_player_labels:
            print('-----------------------------', pl, '-----------------------------')
            analyze_single_player(original_player_labels[pl], redone_player_labels[pl], table_info)
        
#     print('---')
#     for pl in original_player_labels:
#         print(pl)
    return table_info

In [139]:
def process_files(original_folder, redone_folder):
    table_info = []

    for subdir, dirs, files in os.walk(redone_folder):
        ind = 1
        for filename in files:
            name_extension = get_name_and_extension(filename)
            name = name_extension['name']
            ext = name_extension['ext'].lower()

            if ext == 'json':
                print(ind, ":", name)
                if os.path.isfile(original_folder+filename):
                    print('----> ', filename)
                    with open(redone_folder+filename, 'r') as redone_label_data:
                        redone_labels = json.load(redone_label_data)
                        with open(original_folder+filename, 'r') as original_label_data:
                            original_labels = json.load(original_label_data)
                            
                            table_info = analyze_labels_by_players(original_labels, redone_labels, table_info)
                else: 
                    continue
                    
            ind += 1

    return table_info

In [140]:
table_info = process_files(original_label_dir, redone_label_dir)

1 : 24_11_2018__19_29_25_0
---->  24_11_2018__19_29_25_0.json
----------------------------- H_T7_P1 -----------------------------
0 [ Explosive Inner Explosive Inner [69123, 69141] [69124, 69140] ]
0 [ Explosive Inner Explosive Inner [69148, 69156] [69149, 69156] ]
0 [ Explosive Inner Explosive Inner [69185, 69198] [69185, 69196] ]
0 [ Explosive Inner Explosive Inner [69365, 69375] [69366, 69374] ]
0 [ Explosive Inner Explosive Inner [69395, 69409] [69395, 69410] ]
0 [ Explosive Inner Explosive Inner [69420, 69438] [69420, 69438] ]
0 [ Explosive Inner Explosive Inner [69518, 69530] [69520, 69530] ]
0 [ Explosive Inner Explosive Inner [69565, 69577] [69565, 69577] ]
0 [ Explosive Inner Explosive Inner [69606, 69620] [69606, 69620] ]
0 [ Explosive Inner Explosive Inner [69754, 69766] [69755, 69764] ]
0 [ Explosive Inner Explosive Inner [69977, 69994] [69977, 69994] ]
0 [ Exploring Exploring [68845, 68866] [68845, 68865] ]
0 [ Exploring Exploring [68923, 68949] [68924, 68949] ]
0 [ Explor

0 [ Explosive Inner Explosive Inner [11456, 11469] [11458, 11470] ]
0 [ Explosive Inner Explosive Inner [11491, 11522] [11491, 11521] ]
0 [ Exploring Exploring [10521, 10532] [10520, 10532] ]
0 [ Exploring Exploring [10842, 10870] [10839, 10870] ]
0 [ Tunneling Pickaxe Tunneling Pickaxe [10597, 10601] [10599, 10601] ]
0 [ Tunneling Pickaxe Tunneling Pickaxe [10654, 10657] [10654, 10657] ]
0 [ Pickaxing for Gold Pickaxing for Gold [10532, 10592] [10532, 10592] ]
0 [ Pickaxing for Gold Pickaxing for Gold [10601, 10654] [10601, 10653] ]
0 [ Pickaxing for Gold Pickaxing for Gold [10657, 10671] [10658, 10671] ]
0 [ Pickaxing for Gold Pickaxing for Gold [10922, 10990] [10922, 10989] ]
0 [ Exploring Exploring [11128, 11145] [11132, 11146] ]
0 [ Exploring Exploring [11170, 11180] [11171, 11179] ]
0 [ Exploring Exploring [11183, 11229] [11184, 11230] ]
0 [ Exploring Exploring [11419, 11437] [11419, 11438] ]
0 [ Exploring Exploring [11447, 11458] [11447, 11458] ]
0 [ Exploring Exploring [11481, 

0 [ Pickaxing for Gold Pickaxing for Gold [13213, 13246] [13215, 13245] ]
2 [ Exploring No [12287, 12314] -1 ]
3 [ No Pickaxing for Gold -1 [11876, 11878] ]
3 [ No Exploring -1 [11957, 11996] ]
3 [ No Pickaxing for Gold -1 [12553, 12557] ]
3 [ No Pickaxing for Gold -1 [12569, 12600] ]
3 [ No Tunneling Pickaxe -1 [13131, 13135] ]
3 [ No Tunneling Pickaxe -1 [13199, 13215] ]
3 [ No Tunneling Pickaxe -1 [13245, 13252] ]
----------------------------- L_T11_P5 -----------------------------
0 [ Exploring Exploring [11574, 11632] [11573, 11632] ]
0 [ Exploring Exploring [11845, 11866] [11851, 11865] ]
0 [ Tunneling Pickaxe Tunneling Pickaxe [11864, 11918] [11866, 11881] ]
0 [ Tunneling Pickaxe Tunneling Pickaxe [11864, 11918] [11885, 11901] ]
0 [ Tunneling Pickaxe Tunneling Pickaxe [11864, 11918] [11905, 11918] ]
0 [ Tunneling Pickaxe Tunneling Pickaxe [11927, 11963] [11928, 11963] ]
0 [ Pickaxing for Gold Pickaxing for Gold [11633, 11718] [11632, 11644] ]
0 [ Pickaxing for Gold Pickaxing for

0 [ Explosive Quick Mine Explosive Quick Mine [13731, 13736] [13731, 13736] ]
0 [ Explosive Quick Mine Explosive Quick Mine [13741, 13746] [13740, 13745] ]
0 [ Explosive Inner Explosive Inner [13779, 13797] [13779, 13797] ]
0 [ Explosive Quick Mine Explosive Quick Mine [14182, 14187] [14182, 14188] ]
0 [ Exploring Exploring [12987, 13017] [12985, 13016] ]
0 [ Exploring Exploring [13232, 13255] [13232, 13256] ]
0 [ Exploring Exploring [13306, 13326] [13304, 13325] ]
0 [ Pickaxing for Gold Pickaxing for Gold [13017, 13133] [13016, 13038] ]
0 [ Pickaxing for Gold Pickaxing for Gold [13017, 13133] [13047, 13133] ]
0 [ Pickaxing for Gold Pickaxing for Gold [13285, 13305] [13284, 13304] ]
0 [ Exploring Exploring [13465, 13487] [13462, 13487] ]
0 [ Exploring Exploring [13700, 13731] [13701, 13731] ]
0 [ Exploring Exploring [13746, 13779] [13745, 13779] ]
0 [ Pickaxing for Gold Pickaxing for Gold [13508, 13610] [13509, 13586] ]
0 [ Pickaxing for Gold Pickaxing for Gold [13508, 13610] [13588, 1

0 [ Explosive Inner Explosive Inner [12269, 12287] [12271, 12286] ]
0 [ Explosive Quick Mine Explosive Quick Mine [12291, 12298] [12292, 12297] ]
0 [ Explosive Quick Mine Explosive Quick Mine [12435, 12441] [12436, 12442] ]
0 [ Explosive Quick Mine Explosive Quick Mine [12457, 12464] [12458, 12463] ]
0 [ Explosive Inner Explosive Inner [12489, 12500] [12490, 12500] ]
0 [ Exploring Exploring [12017, 12030] [12015, 12029] ]
0 [ Exploring Exploring [12141, 12150] [12142, 12150] ]
0 [ Pickaxing for Gold Pickaxing for Gold [12149, 12166] [12149, 12166] ]
0 [ Pickaxing for Gold Pickaxing for Gold [12063, 12142] [12064, 12142] ]
0 [ Exploring Exploring [12214, 12228] [12214, 12227] ]
0 [ Exploring Exploring [12253, 12270] [12253, 12271] ]
0 [ Pickaxing for Gold Pickaxing for Gold [12304, 12319] [12303, 12319] ]
0 [ Pickaxing for Gold Pickaxing for Gold [12332, 12363] [12332, 12364] ]
0 [ Exploring Exploring [12420, 12436] [12420, 12436] ]
0 [ Exploring Exploring [12441, 12457] [12442, 12458] 

In [141]:
def create_table(info, output_file):
    write_text = []
    write_text.append(heading)

    for inf in info:
        line = []
#         print(inf)
        for l_title in titles:
            if inf[0] == l_title:
                line.append(1)
            else: 
                line.append(0)
        
        line.append(inf[2])
        for l_title in titles:
            if inf[1] == l_title:
                line.append(1)
            else: 
                line.append(0)
        
        line.append(inf[3])
        
#         print(line)
        write_text.append(line)
    
    
    with open(output_file, 'w') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(write_text)

In [142]:
create_table(table_info, irr_table_output_file)

In [143]:
column_names = []
index_names = []
heading_len = len(heading)
for i in range(0, int(heading_len/2)-1):
#     print(redone_label_initial+' '+heading[i])
    column_names.append(redone_label_initial+' '+heading[i])
#     print(orginal_label_initial+' '+heading[i+int(heading_len/2)])
    index_names.append(orginal_label_initial+' '+heading[i+int(heading_len/2)])

In [144]:
with open(irr_table_output_file) as csv_file:
    irr_table = pd.read_csv(csv_file)

# print(irr_table)

In [145]:
row, column = irr_table.shape
half_column = int(column/2)

In [146]:
irr_matrix = []
for i in range (half_column, column-1):
    matrix_row = []
    for j in range(0, half_column-1):
        cmp_df = irr_table.iloc[:,[i,j]]
        cmp_df.columns = ['one', 'two']
        matrix_row.append(np.sum((cmp_df['one'] == 1) & (cmp_df['two'] == 1)))
    irr_matrix.append(matrix_row)
    

In [147]:
matrix_df = pd.DataFrame(np.array(irr_matrix), columns=column_names, index=index_names)
export_csv = matrix_df.to_csv(irr_matrix_output_file, header=True)

In [148]:
matrix_df

,Jimmy Explosive Quick Mine,Jimmy Explosive Inner,Jimmy Explosive Tunneling,Jimmy Exploring,Jimmy Tunneling Pickaxe,Jimmy Pickaxing for Gold,Jimmy Explosive Miss,Jimmy No Label
Erica Explosive Quick Mine,80,9,7,0,0,0,0,6
Erica Explosive Inner,9,139,1,0,0,1,0,1
Erica Explosive Tunneling,0,0,20,0,0,0,0,0
Erica Exploring,0,0,0,342,0,1,0,21
Erica Tunneling Pickaxe,0,1,0,0,95,0,0,85
Erica Pickaxing for Gold,0,8,0,0,0,355,0,108
Erica Explosive Miss,0,0,1,0,0,0,3,1
Erica No Label,6,1,0,33,6,3,1,0


In [149]:
sum_mat_df = matrix_df.copy()
# sum_mat_df['sum'] = sum_mat_df.sum(axis=1)
# sum_mat_df

# sum_mat_df = matrix_df.copy()
# #Total sum per row: 
# sum_mat_df.loc['SUM',:]= sum_mat_df.sum(axis=0)
# #Total sum per column: 
# sum_mat_df.loc[:,'SUM'] = sum_mat_df.sum(axis=1)
# sum_mat_df

sum_mat_df.append(sum_mat_df.sum().rename('SUM')).assign(SUM=lambda d: d.sum(1))

,Jimmy Explosive Quick Mine,Jimmy Explosive Inner,Jimmy Explosive Tunneling,Jimmy Exploring,Jimmy Tunneling Pickaxe,Jimmy Pickaxing for Gold,Jimmy Explosive Miss,Jimmy No Label,SUM
Erica Explosive Quick Mine,80,9,7,0,0,0,0,6,102
Erica Explosive Inner,9,139,1,0,0,1,0,1,151
Erica Explosive Tunneling,0,0,20,0,0,0,0,0,20
Erica Exploring,0,0,0,342,0,1,0,21,364
Erica Tunneling Pickaxe,0,1,0,0,95,0,0,85,181
Erica Pickaxing for Gold,0,8,0,0,0,355,0,108,471
Erica Explosive Miss,0,0,1,0,0,0,3,1,5
Erica No Label,6,1,0,33,6,3,1,0,50
SUM,95,158,29,375,101,360,4,222,1344


In [150]:
def calculate_cohen_kappa(conf_mat, exclude=None):
    cm = conf_mat.copy()
    if exclude:
        cm = cm.drop(exclude[0], axis=1)
        cm = cm.drop(exclude[1], axis=0)
#     print(cm)
    
    cm = np.array(cm)
    sum_total = np.sum(cm)
    sum_row = np.sum(cm, axis=1)
    sum_column = np.sum(cm, axis=0)
    p_c = np.sum(np.multiply(sum_row, sum_column))/(sum_total**2)
    p_o = np.trace(cm)/sum_total
    k = (p_o - p_c)/(1 - p_c)
    
    return k

In [151]:
k = calculate_cohen_kappa(matrix_df)
print(k)

0.710011574759


In [152]:
k = calculate_cohen_kappa(matrix_df, exclude=[['Jimmy No Label'],['Erica No Label']])
print(k)

0.952659036051


In [116]:
k = calculate_cohen_kappa(matrix_df, exclude=[['Jimmy No Label','Jimmy Exploring'],['Erica No Label','Erica Exploring']])
print(k)

0.925159611217


In [117]:
k = calculate_cohen_kappa(matrix_df, exclude=[['Jimmy Exploring'],['Erica Exploring']])
print(k)

0.640579930165
